In [1]:
!guardrails hub install hub://brainlogic/high_quality_translation

Installing hub://brainlogic/high_quality_translation...
[ ===] Fetching manifestst
[=   ] Downloading dependenciespendencies  Running command git clone --filter=blob:none --quiet https://github.com/BrainLogicHub/high_quality_translation_validator.git /private/var/folders/yt/ltz0vbpx14j34mj55jyj39x40000gn/T/pip-req-build-hja54mr_
[   =] Downloading dependenciesWARNING: typer 0.12.5 does not provide the extra 'all'
[=== ] Downloading dependencies
Fetching 5 files: 100%|████████████████████████| 5/5 [00:00<00:00, 45590.26it/s]
[ ===] Running post-install setupLightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
[=   ] Running post-install setup/Users/dtam/.pyenv/versions/3.11.3/envs/311lite/lib/python3.11/site-p

# Translate text with quality checks

**Note:**
To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/translation_with_quality_check.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to English. We will check whether the translated statement is likely of high quality.

## Objective

We want to translate a statement from different languages to English and ensure that the translated statement accurately reflects the original content.

### Setup

- Install the `unbabel-comet` from source:
  `pip install git+https://github.com/Unbabel/COMET`
- Please accept the model license from:
  https://huggingface.co/Unbabel/wmt22-cometkiwi-da
- Login into Huggingface Hub using:
  huggingface-cli login --token $HUGGINGFACE_TOKEN


In [2]:
from guardrails import Guard
from rich import print
from guardrails.hub import HighQualityTranslation

## Step 1: Define a Guard that uses the validator

This guard will use the HighQualityTranslation validator to validate some string outputs.


We define the prompt and the guard.

In [3]:
prompt = """
Translate the given statement into English:

${statement_to_be_translated}
"""

guard = Guard().use(HighQualityTranslation(on_fail="fix"))

Loading the model Unbabel/wmt22-cometkiwi-da...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
/Users/dtam/.pyenv/versions/3.11.3/envs/311lite/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Encoder model frozen.
/Users/dtam/.pyenv/versions/3.11.3/envs/311lite/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.po

## Step 2: Wrap the LLM API call with `Guard`


First, let's try translating a statement that is relatively easy to translate.


In [4]:
# Set your OPENAI_API_KEY as an environment variable
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

statement = "Ich habe keine Ahnung, was ich hier schreiben soll."

res = guard(
    messages=[{"role": "user", "content": prompt}],
    prompt_params={"statement_to_be_translated": statement},
    metadata={"translation_source": statement},
    model="gpt-3.5-turbo",
    max_tokens=1024,
    temperature=0,
)

print(f"Raw LLM Output: {res.raw_llm_output}")
print(f"Validated Output: {res.validated_output}")

HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
10:09:59 - LiteLLM:INFO: utils.py:2963 - 
LiteLLM completion() model= gpt-3.5-turbo; provider = openai

LiteLLM completion() model= gpt-3.5-turbo; provider = openai
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
10:09:59 - LiteLLM:INFO: utils.py:989 - Wrapper: Completed Call, calling success_handler
Wrapper: Completed Call, calling success_handler
/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
/Users/dtam/.pyenv/versions/3.11.3/envs/311lite/lib/python3.11/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_se

Raw LLM Output: I have no idea what I should write here.

Validated Output: I have no idea what I should write here.

We can look at the logs to see the quality check results:


In [5]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓                                          │ │
    │ │ ┃ Role ┃ Content                                             ┃                                          │ │
    │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩                                          │ │
    │ │ │ user │                                                     │                                          │ │
    │ │ │      │ Translate the given statement into English:         │                                          │ │
    │ │ │      │                                                     │                                          │ │
    │ │ │      │ Ich habe keine Ahnung, was ich hier schreiben soll. │                                          │ │
    │ │ │      │                                                     │                                          │ │
    │ │ └──────┴─────────────────────────────────────────────────────┘                                          │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ I have no idea what I should write here.                                                                │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ I have no idea what I should write here.                                                                │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The `guard` wrapper returns the raw LLM response, which is the translated statement and also the validated output. In this case, the translated statement was of a good quality (above the threshold of 0.5), so the validated output is the same as the raw LLM response.

#### Now, let's test with a really low quality translation, and see how Guardrails handles it.


In [6]:
# Parse the code snippet
statement = "अरे भाऊ, आज रात्री जोरदार पार्टी मारूया, जमून टाकूया आणि धमाल करूया!"

## Ideal translation from Marathi -> English:
#  "Hey bro, let's have a great party tonight and have fun!"

output = guard.parse(
    llm_output="It's such a beautiful day, I'm going to the beach.",  ## here, providing a really bad translation
    metadata={"translation_source": statement},
)

# Print the output
print(f"Raw LLM Output: {output.raw_llm_output}")
print(f"Validated Output: {output.validated_output}")

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/dtam/.pyenv/versions/3.11.3/envs/311lite/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/dtam/.pyenv/versions/3.11.3/envs/311lite/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 15.53it/s]


Raw LLM Output: It's such a beautiful day, I'm going to the beach.

Validated Output:

In [7]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ No messages.                                                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ It's such a beautiful day, I'm going to the beach.                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ ''                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the translation quality is really bad, and the `HighQualityTranslation` check failed as the translation quality was below the threshold. The validated response is an empty string.

## In this way, you can use Guardrails to ensure that the output of your LLM is of high quality.
